## Robot Run

Installation:
* https://www.anaconda.com/download/
* git clone git@github.com:DJCordhose/haw.git
* cd haw/notebooks
* jupyter notebook

Or clone on Azure Notebooks
* https://notebooks.azure.com/djcordhose/libraries/ai-haw

In a certain terrain a Robot (R) has to navigate to a Goal (G) avoiding Blocks (B) and human player (H)

In [1]:
terrain = [
    ["R", "_", "_", "_"],
    ["_", "_", "B", "_"],
    ["H", "_", "B", "_"],
    ["B", "_", "G", "_"]   
]

In [2]:
from copy import deepcopy

robot_symbol = 'R'
robot_win_symbol = '*'
goal_symbol = 'G'
human_symbol = 'H'
blank_symbol = '_'

def is_robot_win(state):
    for row in state:
        for field in row:
            if field == robot_win_symbol:
                return True
    return False   

def as_string(state):
    s = ''
    for row in state:
        row_string = ''
        for field in row:
            row_string += field + ' '
        s += row_string + '\n'
    return s

def locate(state, what):
    for row_index, row in enumerate(state):
        for column_index, field in enumerate(row):
            if field == what:
                return (row_index, column_index)

def check_robot(state, robot):
    max_row = len(state) - 1
    max_column = len(state[0]) - 1
    if robot[0] < 0 or robot[0] > max_row or robot[1] < 0 or robot[1] > max_column:
        return False
    symbol = state[robot[0]][robot[1]]
    if symbol != blank_symbol and symbol != goal_symbol:
        return False
    return True
            
def robot_moves(state):
    robot = locate(state, robot_symbol)
    left = (robot[0], robot[1] - 1)
    right = (robot[0], robot[1] + 1)
    up = (robot[0] - 1, robot[1])
    down = (robot[0] + 1, robot[1])
    valid_moves = [move for move in (left, right, down, up) if check_robot(state, move)]
    return valid_moves
            
def place_robot(state, robot):
    old_robot = locate(state, robot_symbol)
    new_state = deepcopy(state)
    new_state[old_robot[0]][old_robot[1]] = blank_symbol
    if new_state[robot[0]][robot[1]] == goal_symbol:
        new_state[robot[0]][robot[1]] = robot_win_symbol
    else:
        new_state[robot[0]][robot[1]] = robot_symbol
    return new_state

def expand(state):
    valid_moves = robot_moves(state)
    new_states = [(robot, place_robot(state, robot)) for robot in valid_moves]
    return new_states

In [3]:
# https://en.wikipedia.org/wiki/Depth-first_search
# 1  procedure DFS(G,v):
# 2      label v as discovered
# 3      for all edges from v to w in G.adjacentEdges(v) do
# 4          if vertex w is not labeled as discovered then
# 5              recursively call DFS(G,w)

def depth_first_search(state, max_depth=10, debug=False, closed_list=[], depth = 0, path=[]):
    if as_string(state) in closed_list or depth > max_depth:
        return None
    
    if debug:
        print('depth', depth)
        print('closed_list', closed_list)
        print('path', path)
        print('state', as_string(state))
        
    if is_robot_win(state):
        return path
    
    closed_list = closed_list + [as_string(state)]
    
    for move, next_state in expand(state):
        new_path = path + [move]
        res = depth_first_search(next_state, max_depth, debug, closed_list, depth + 1, new_path)
        if res:
            return res

In [4]:
# this does not find the best path
depth_first_search(terrain, 7)

[(0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (3, 2)]

In [5]:
# if we restrict ourselves to only five levels we find the best
depth_first_search(terrain, 5)

[(0, 1), (1, 1), (2, 1), (3, 1), (3, 2)]

## Understanding the search in a simplified terrain

In [6]:
simple_terrain = [
    ["R", "_", "_", "_"],
    ["B", "_", "G", "_"] 
]

In [7]:
depth_first_search(simple_terrain, debug=True)

depth 0
closed_list []
path []
state R _ _ _ 
B _ G _ 

depth 1
closed_list ['R _ _ _ \nB _ G _ \n']
path [(0, 1)]
state _ R _ _ 
B _ G _ 

depth 2
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n']
path [(0, 1), (0, 2)]
state _ _ R _ 
B _ G _ 

depth 3
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n']
path [(0, 1), (0, 2), (0, 3)]
state _ _ _ R 
B _ G _ 

depth 4
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n', '_ _ _ R \nB _ G _ \n']
path [(0, 1), (0, 2), (0, 3), (1, 3)]
state _ _ _ _ 
B _ G R 

depth 5
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n', '_ _ _ R \nB _ G _ \n', '_ _ _ _ \nB _ G R \n']
path [(0, 1), (0, 2), (0, 3), (1, 3), (1, 2)]
state _ _ _ _ 
B _ * _ 



[(0, 1), (0, 2), (0, 3), (1, 3), (1, 2)]

In [8]:
depth_first_search(simple_terrain, max_depth=3, debug=True)

depth 0
closed_list []
path []
state R _ _ _ 
B _ G _ 

depth 1
closed_list ['R _ _ _ \nB _ G _ \n']
path [(0, 1)]
state _ R _ _ 
B _ G _ 

depth 2
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n']
path [(0, 1), (0, 2)]
state _ _ R _ 
B _ G _ 

depth 3
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n']
path [(0, 1), (0, 2), (0, 3)]
state _ _ _ R 
B _ G _ 

depth 3
closed_list ['R _ _ _ \nB _ G _ \n', '_ R _ _ \nB _ G _ \n', '_ _ R _ \nB _ G _ \n']
path [(0, 1), (0, 2), (1, 2)]
state _ _ _ _ 
B _ * _ 



[(0, 1), (0, 2), (1, 2)]

## Can we find the best path without manually restricting the depth?

In [9]:
# https://en.wikipedia.org/wiki/Breadth-first_search
def breadth_first_search(root, max_depth=10, debug=False):
    closed_list = set()
    open_list = []
    meta={}
    
    meta[as_string(root)] = (None, None, 0)
    open_list.append(root)
    
    while open_list:
        if debug:
            print('open_list', open_list)
            print('closed_list', closed_list)
            
        state = open_list.pop()
        depth = meta[as_string(state)][2]
        
        if debug:
            print('expanding level', depth)
            print(as_string(state))

        if is_robot_win(state):
            path = construct_path(as_string(state), meta)
            return path

        if depth >= max_depth:
            continue
        
        for action, child in expand(state):
            if as_string(child) in closed_list:
                continue
                
            if child not in open_list:
                meta[as_string(child)] = (as_string(state), action, depth + 1)
                if debug:
                    print('adding to open list', (child, depth + 1))
                open_list.insert(0, child)

        closed_list.add(as_string(state))
        
def construct_path(state, meta):
  path = []
  
  while True:
    row = meta[state]
    if row[0]:
      state = row[0]
      action = row[1]
      path.insert(0, action)
    else:
      break
  
  return path

In [10]:
breadth_first_search(simple_terrain, max_depth = 3, debug=True)

open_list [[['R', '_', '_', '_'], ['B', '_', 'G', '_']]]
closed_list set()
expanding level 0
R _ _ _ 
B _ G _ 

adding to open list ([['_', 'R', '_', '_'], ['B', '_', 'G', '_']], 1)
open_list [[['_', 'R', '_', '_'], ['B', '_', 'G', '_']]]
closed_list {'R _ _ _ \nB _ G _ \n'}
expanding level 1
_ R _ _ 
B _ G _ 

adding to open list ([['_', '_', 'R', '_'], ['B', '_', 'G', '_']], 2)
adding to open list ([['_', '_', '_', '_'], ['B', 'R', 'G', '_']], 2)
open_list [[['_', '_', '_', '_'], ['B', 'R', 'G', '_']], [['_', '_', 'R', '_'], ['B', '_', 'G', '_']]]
closed_list {'_ R _ _ \nB _ G _ \n', 'R _ _ _ \nB _ G _ \n'}
expanding level 2
_ _ R _ 
B _ G _ 

adding to open list ([['_', '_', '_', 'R'], ['B', '_', 'G', '_']], 3)
adding to open list ([['_', '_', '_', '_'], ['B', '_', '*', '_']], 3)
open_list [[['_', '_', '_', '_'], ['B', '_', '*', '_']], [['_', '_', '_', 'R'], ['B', '_', 'G', '_']], [['_', '_', '_', '_'], ['B', 'R', 'G', '_']]]
closed_list {'_ R _ _ \nB _ G _ \n', 'R _ _ _ \nB _ G _ \

[(0, 1), (0, 2), (1, 2)]

In [11]:
breadth_first_search(terrain)

[(0, 1), (1, 1), (2, 1), (3, 1), (3, 2)]

In [12]:
depth_first_search(terrain, 5)

[(0, 1), (1, 1), (2, 1), (3, 1), (3, 2)]

## Is there a more general version?

In [13]:
def search(root, generate_open_list):
    closed_list = set()
    open_list = []
    meta={}
    
    meta[as_string(root)] = (None, None, 0, 0)
    open_list.append(root)
    
    while open_list:
        state = open_list.pop()
        
        if is_robot_win(state):
            path = construct_path(as_string(state), meta)
            print('nodes visited', len(closed_list))
            return path

        expanded = expand(state)
        open_list = generate_open_list(state, expanded, open_list, closed_list, meta)
        closed_list.add(as_string(state))

In [14]:
max_depth=10
def depth_first_generator(parent, children, open_list, closed_list, meta):
    if debug:
        print('open_list', open_list)
        print('closed_list', closed_list)
            
    new_open_list = list(open_list)

    depth = meta[as_string(parent)][2]
    
    if depth < max_depth:
        if debug:
            print('expanding level', depth)
            print(as_string(parent))
            
        for action, child in children:
            if as_string(child) in closed_list:
                continue

            if child not in open_list:
                meta[as_string(child)] = (as_string(parent), action, depth + 1)
                new_open_list.append(child)
    return new_open_list

In [15]:
# this might already find the best path, but it is not guaranteed at all
debug=False
search(simple_terrain, depth_first_generator)

nodes visited 3


[(0, 1), (1, 1), (1, 2)]

In [16]:
debug=False
search(terrain, depth_first_generator)

nodes visited 5


[(1, 0), (1, 1), (2, 1), (3, 1), (3, 2)]

In [17]:
max_depth=10
def breadth_first_generator(parent, children, open_list, closed_list, meta):
    if debug:
        print('open_list', open_list)
        print('closed_list', closed_list)
            
    new_open_list = list(open_list)

    depth = meta[as_string(parent)][2]
    
    if depth < max_depth:
        if debug:
            print('expanding level', depth)
            print(as_string(parent))
            
        for action, child in children:
            if as_string(child) in closed_list:
                continue

            if child not in open_list:
                meta[as_string(child)] = (as_string(parent), action, depth + 1)
                new_open_list.insert(0, child)
    return new_open_list

In [18]:
debug=False
search(simple_terrain, breadth_first_generator)

nodes visited 5


[(0, 1), (0, 2), (1, 2)]

In [19]:
debug=False
search(terrain, breadth_first_generator)

nodes visited 10


[(0, 1), (1, 1), (2, 1), (3, 1), (3, 2)]

## Why do we blindly wander around, don't we know in which direction to walk?
The closed list shows the number of traversed nodes, and there are quite a few

In [20]:
# https://en.wikipedia.org/wiki/A*_search_algorithm

max_depth=10
def a_star_generator(parent, children, open_list, closed_list, meta):
    if debug:
        print('open_list', open_list)
        print('closed_list', closed_list)
            
    new_open_list = list(open_list)

    depth = meta[as_string(parent)][2]
    previous_cost = depth
    
    if depth < max_depth:
        if debug:
            print('expanding level', depth)
            print(as_string(parent))
            
        for action, child in children:
            if as_string(child) in closed_list:
                continue

            if child not in open_list:
                estimated_rest_cost = estimate_rest_cost(child)
                estimated_total_cost = estimated_rest_cost + previous_cost
                meta[as_string(child)] = (as_string(parent), action, depth + 1, estimated_total_cost)
                new_open_list.append(child)
    
        # sort using total cost, expand least total cost first
        new_open_list.sort(key=lambda x: meta[as_string(child)][3])
    return new_open_list

In [21]:
# https://en.wikipedia.org/wiki/Admissible_heuristic
# we must not overestimate, which is called "admissible"
# strangely enough, this is admissible, but obviously not perfect
def estimate_rest_cost(child):
    return 1

debug=False
search(simple_terrain, a_star_generator)

nodes visited 3


[(0, 1), (1, 1), (1, 2)]

In [22]:
search(terrain, a_star_generator)

nodes visited 5


[(1, 0), (1, 1), (2, 1), (3, 1), (3, 2)]

## A better heuristic
Best of both worlds: always findes best solution, but visited nodes are minimal

In [23]:
from math import sqrt, pow

def distance(pos1, pos2):
    if pos1 and pos2:
        return sqrt(pow(pos1[0] - pos2[0], 2) + pow(pos1[1] - pos2[1], 2))
    else:
        return 0

def estimate_rest_cost(child):
    robot = locate(child, robot_symbol)
    goal = locate(child, goal_symbol)
    if debug:
        print('robot', robot)
        print('goal', goal)
        print('distance', distance(robot, goal))
    return distance(robot, goal)

debug=False
search(simple_terrain, a_star_generator)

nodes visited 3


[(0, 1), (1, 1), (1, 2)]

In [24]:
simple_terrain

[['R', '_', '_', '_'], ['B', '_', 'G', '_']]

In [25]:
debug=True
search(simple_terrain, a_star_generator)

open_list []
closed_list set()
expanding level 0
R _ _ _ 
B _ G _ 

robot (0, 1)
goal (1, 2)
distance 1.4142135623730951
open_list []
closed_list {'R _ _ _ \nB _ G _ \n'}
expanding level 1
_ R _ _ 
B _ G _ 

robot (0, 2)
goal (1, 2)
distance 1.0
robot (1, 1)
goal (1, 2)
distance 1.0
open_list [[['_', '_', 'R', '_'], ['B', '_', 'G', '_']]]
closed_list {'_ R _ _ \nB _ G _ \n', 'R _ _ _ \nB _ G _ \n'}
expanding level 2
_ _ _ _ 
B R G _ 

robot None
goal None
distance 0
nodes visited 3


[(0, 1), (1, 1), (1, 2)]

In [26]:
debug=False
search(terrain, a_star_generator)

nodes visited 5


[(1, 0), (1, 1), (2, 1), (3, 1), (3, 2)]